In [1]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark = SparkSession. \
builder. \
config('spark.ui.port', '0'). \
config('spark.shuffle.useOldFetchProtocol', 'true'). \
config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

In [2]:
orders_schema = "order_id long , order_date string, customer_id long,order_status string"

In [3]:
orders_df = spark.read \
.format("csv") \
.schema(orders_schema) \
.load("/public/trendytech/orders/orders_1gb.csv")

In [4]:
customer_schema = " customerid long , customer_fname string , customer_lname string , user_name string,password string , address string, city string, state string, pincode long "

In [5]:
customers_df = spark.read \
.format("csv") \
.schema(customer_schema) \
.load("/public/trendytech/retail_db/customers")

In [6]:
spark.conf.get('spark.sql.autoBroadcastJoinThreshold')

'10485760b'

In [7]:
orders_df.join(customers_df, orders_df.customer_id == customers_df.customerid, "inner").write.format("noop").mode("overwrite").save()

In [8]:
orders_df.join(customers_df, orders_df.customer_id == customers_df.customerid, "inner").explain()

== Physical Plan ==
*(2) BroadcastHashJoin [customer_id#2L], [customerid#8L], Inner, BuildRight, false
:- *(2) Filter isnotnull(customer_id#2L)
:  +- FileScan csv [order_id#0L,order_date#1,customer_id#2L,order_status#3] Batched: false, DataFilters: [isnotnull(customer_id#2L)], Format: CSV, Location: InMemoryFileIndex[hdfs://m01.itversity.com:9000/public/trendytech/orders/orders_1gb.csv], PartitionFilters: [], PushedFilters: [IsNotNull(customer_id)], ReadSchema: struct<order_id:bigint,order_date:string,customer_id:bigint,order_status:string>
+- BroadcastExchange HashedRelationBroadcastMode(List(input[0, bigint, false]),false), [id=#65]
   +- *(1) Filter isnotnull(customerid#8L)
      +- FileScan csv [customerid#8L,customer_fname#9,customer_lname#10,user_name#11,password#12,address#13,city#14,state#15,pincode#16L] Batched: false, DataFilters: [isnotnull(customerid#8L)], Format: CSV, Location: InMemoryFileIndex[hdfs://m01.itversity.com:9000/public/trendytech/retail_db/customers], Partitio

In [9]:
spark.conf.set('spark.sql.autoBroadcastJoinThreshold', '-1')

In [10]:
orders_df.join(customers_df, orders_df.customer_id == customers_df.customerid, "inner").write.format("noop").mode("overwrite").save()

In [11]:
orders_df.join(customers_df, orders_df.customer_id == customers_df.customerid, "inner").explain()

== Physical Plan ==
*(5) SortMergeJoin [customer_id#2L], [customerid#8L], Inner
:- *(2) Sort [customer_id#2L ASC NULLS FIRST], false, 0
:  +- Exchange hashpartitioning(customer_id#2L, 200), ENSURE_REQUIREMENTS, [id=#149]
:     +- *(1) Filter isnotnull(customer_id#2L)
:        +- FileScan csv [order_id#0L,order_date#1,customer_id#2L,order_status#3] Batched: false, DataFilters: [isnotnull(customer_id#2L)], Format: CSV, Location: InMemoryFileIndex[hdfs://m01.itversity.com:9000/public/trendytech/orders/orders_1gb.csv], PartitionFilters: [], PushedFilters: [IsNotNull(customer_id)], ReadSchema: struct<order_id:bigint,order_date:string,customer_id:bigint,order_status:string>
+- *(4) Sort [customerid#8L ASC NULLS FIRST], false, 0
   +- Exchange hashpartitioning(customerid#8L, 200), ENSURE_REQUIREMENTS, [id=#157]
      +- *(3) Filter isnotnull(customerid#8L)
         +- FileScan csv [customerid#8L,customer_fname#9,customer_lname#10,user_name#11,password#12,address#13,city#14,state#15,pincode#16

In [12]:
orders_df.join(customers_df.hint("shuffle_hash"), orders_df.customer_id == customers_df.customerid, "inner").write.format("noop").mode("overwrite").save()

In [13]:
orders_df.join(customers_df.hint("shuffle_hash"), orders_df.customer_id == customers_df.customerid, "inner").explain()

== Physical Plan ==
*(3) ShuffledHashJoin [customer_id#2L], [customerid#8L], Inner, BuildRight
:- Exchange hashpartitioning(customer_id#2L, 200), ENSURE_REQUIREMENTS, [id=#234]
:  +- *(1) Filter isnotnull(customer_id#2L)
:     +- FileScan csv [order_id#0L,order_date#1,customer_id#2L,order_status#3] Batched: false, DataFilters: [isnotnull(customer_id#2L)], Format: CSV, Location: InMemoryFileIndex[hdfs://m01.itversity.com:9000/public/trendytech/orders/orders_1gb.csv], PartitionFilters: [], PushedFilters: [IsNotNull(customer_id)], ReadSchema: struct<order_id:bigint,order_date:string,customer_id:bigint,order_status:string>
+- Exchange hashpartitioning(customerid#8L, 200), ENSURE_REQUIREMENTS, [id=#239]
   +- *(2) Filter isnotnull(customerid#8L)
      +- FileScan csv [customerid#8L,customer_fname#9,customer_lname#10,user_name#11,password#12,address#13,city#14,state#15,pincode#16L] Batched: false, DataFilters: [isnotnull(customerid#8L)], Format: CSV, Location: InMemoryFileIndex[hdfs://m01.it